In [ ]:
import os
import ntpath
import shutil
import sys
import argparse
from datetime import datetime
import random

import numpy as np
import cv2
from PIL import Image, ImageChops, ImageDraw, ImageOps, ImageFilter, ImageStat, ImageEnhance 
from skimage import io, color, exposure
#import imutils
import matplotlib
import matplotlib.pyplot as plt
import glob
import math
#import blend_modes

from damage import no_damage, remove_quadrant, remove_hole, bullet_holes, graffiti, bend_vertical
from utils import load_paths, load_files

original = True # Whether we are using the original exposure_manipulation code or not

In [ ]:
def dir_split(path):
    """Imitates the functionality of path.split('/') while using os.path.split"""
    # https://stackoverflow.com/a/3167684/12350950
    folders = []
    while True:
        path, folder = os.path.split(path)
        if folder != "":
            folders.append(folder)
        else:
            if path != "":
                folders.append(path)
            break

    folders.reverse()

    return folders

In [ ]:
def scale_image(image_path):
    """Rescales and pads the source image with whitespace to be a perfect square of fixed width"""
    width = 200

    # https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/
    # Resize the image
    img = Image.open(image_path)
    old_size = img.size  # old_size is in (width, height) format
    ratio = float(width) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    img = img.resize(new_size, Image.ANTIALIAS)

    # Pad the image with whitespace
    delta_w = width - new_size[0]
    delta_h = width - new_size[1]
    padding = (delta_w // 2, delta_h // 2, delta_w - (delta_w // 2), delta_h - (delta_h // 2))
    new_img = ImageOps.expand(img, padding, (255,255,255))
    
    return new_img

In [ ]:
def create_alpha(img, alpha_channel):
    """Returns an alpha channel that matches the white background \n
    Based on Alexandros Stergiou's find_borders() function"""

    # Read and decode the image contents for pixel access
    pix = img.load()

    # Note PIL indexes [x,y] while OpenCV indexes [y,x]
    # alpha_channel must be indexed [y,x] to merge with OpenCV channels later

    min = 200
    width, height = img.size
    # Loop through each row of the image
    for y in range(0, height):
        # First loop left to right
        for x in range(0, width, 1):
            # Retrieve a tuple with RGB values for this pixel
            rgb = pix[x,y]
            # Make transparent if the pixel is white (or light enough)
            if rgb[0] >= min and rgb[1] >= min and rgb[2] >= min:
                alpha_channel[y,x] = 0
            # If pixel is not white then we've hit the sign so break out of loop
            else:
                break

        # Then loop backwards, right to left
        for x in range(width-1, -1, -1):
            rgb = pix[x,y]
            if rgb[0] >= min and rgb[1] >= min and rgb[2] >= min:
                alpha_channel[y,x] = 0
            else:
                break

    return alpha_channel

In [ ]:
def delete_background(image_path, save_path):
    """Deletes the white background from the original sign
    Based on Alexandros Stergiou's manipulate_images() function"""
    # Open the image using PIL (don't read contents yet)
    img = Image.open(image_path)
    img = img.convert('RGB')  # Does this have any effect??

    # Open the image again using OpenCV and split into its channels
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    channels = cv2.split(image)

    # Create a fully opaque alpha channel, same dimentions and dtype as the image
    # create_alpha() modifies it to make the white background transparent
    alpha_channel = np.ones(channels[0].shape, dtype=channels[0].dtype) * 255
    alpha_channel = create_alpha(img, alpha_channel)

    # Merge alpha channel into original image
    image_RGBA = cv2.merge((channels[0], channels[1], channels[2], alpha_channel))

    cv2.imwrite(save_path, image_RGBA)

    img.close()

In [ ]:
# Create the output directories if they don't exist already
base_dir = "Traffic_Signs_Templates"
input_dir = os.path.join(base_dir, "1_Input_Images")
output_dir = os.path.join(base_dir, "2_Processed_Images")
if (not os.path.exists(base_dir)):
    os.mkdir(base_dir)
if (not os.path.exists(output_dir)):
    os.mkdir(output_dir)

# Rescale images and make white backgrounds transparent
paths = load_files(input_dir)
for path in paths:
    img = scale_image(path) # Rescale the image

    # Remove the extension and save as a png
    _, filename = ntpath.split(path)
    name, _ = filename.rsplit('.', 1)
    save_path = os.path.join(output_dir, name) + ".png"
    img.save(save_path)

    delete_background(save_path, save_path) # Overwrite the newly rescaled image

In [ ]:
# Used for SGTSD
damage_types = ["ORIGINAL", "QUADRANT", "BIG_HOLE", "HOLES", "YELLOW", #TODO: Remove damage_types and fix downstream dependencies
                 "GRAFFITI", "BEND", "GREY"]
labels_dir = "SGTSD/Labels"

def damage_image(image_path, output_dir):
    """Applies all the different types of damage to the imported image, saving each one"""
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    img = img.astype('uint8')
    
    # Create file writing info: filename, class number, output directory, and labels directory
    _, filename = ntpath.split(image_path)  # Remove parent directories to retrieve the image filename
    class_num, _ = filename.rsplit('.', 1)  # Remove extension to get the sign/class number

    output_path = os.path.join(output_dir, class_num)
    # Create the output directory if it doesn't exist already
    if (not os.path.exists(output_path)):
        os.makedirs(output_path)

    # Create the labels directory for this sign if it doesn't exist already
    if (not os.path.exists(os.path.join(labels_dir, class_num))):
        os.makedirs(os.path.join(labels_dir, class_num))
    # text_base = os.path.join(labels_dir, class_num, class_num + "_")  # The base filename for the label files
    # ii = 0  # Class number that is appended to the end of base filename


    # ORIGINAL UNDAMAGED
    # dmg_values = calc_quadrant_diff(dmg, img)
    # values = [float_to_string(round(num, 6)) for num in dmg_values]  # Convert the values into decimal strings
    # text_file = open(text_base + str(ii) + ".txt", "w")
    # text_file.write(f"{values[0]} {values[1]} {values[2]} {values[3]}")
    # ii += 1
    # cv2.imwrite(os.path.join(output_path, class_num + ".png"), dmg)
    dmg, att = no_damage(img)
    cv2.imwrite(os.path.join(output_path, class_num + ".png"), dmg)
    
    # QUADRANT
    # dmg_values = calc_quadrant_diff(dmg1, img)
    # values = [float_to_string(round(num, 6)) for num in dmg_values]  # Convert the values into decimal strings
    # text_file = open(text_base + str(ii) + ".txt", "w")
    # text_file.write(f"{values[0]} {values[1]} {values[2]} {values[3]}")
    # ii += 1
    # cv2.imwrite(os.path.join(output_path, class_num + damage_types[1] + ".png"), dmg1)
    dmg1, att = remove_quadrant(img)
    cv2.imwrite(os.path.join(output_path, class_num + "_" + att["damage"] + ".png"), dmg1)
    
    # BIG HOLE
    # dmg_values = calc_quadrant_diff(dmg2, img)
    # values = [float_to_string(round(num, 6)) for num in dmg_values]  # Convert the values into decimal strings
    # text_file = open(text_base + str(ii) + ".txt", "w")
    # text_file.write(f"{values[0]} {values[1]} {values[2]} {values[3]} angle={angle}")
    # ii += 1
    # cv2.imwrite(os.path.join(output_path, class_num + damage_types[2] + ".png"), dmg2)
    dmg2, att = remove_hole(img)
    cv2.imwrite(os.path.join(output_path, class_num + "_" + att["damage"] + ".png"), dmg2)
    
    # RANDOMISED BULLET HOLES
    # dmg_values = calc_quadrant_diff(dmg3, img)
    # values = [float_to_string(round(num, 6)) for num in dmg_values]  # Convert the values into decimal strings
    # text_file = open(text_base + str(ii) + ".txt", "w")
    # text_file.write(f"{values[0]} {values[1]} {values[2]} {values[3]} holes={numHoles}")
    # ii += 1
    # cv2.imwrite(os.path.join(output_path, class_num + damage_types[3] + ".png"), dmg3)
    dmg3, att = bullet_holes(img)
    cv2.imwrite(os.path.join(output_path, class_num + "_" + att["damage"] + ".png"), dmg3)
    
    # TINTED YELLOW
    # yellow = np.zeros((height,width,ch), dtype=np.uint8)
    # yellow[:,:] = (0,210,210,255)
    # dmg4 = cv2.bitwise_and(img, yellow)
    # # TODO: Use quadrant pixel difference ratio or some other damage metric for labelling?
    # cv2.imwrite(os.path.join(output_path, class_num + damage_types[4] + ".png"), dmg4)
    
    # GRAFFITI
    # dmg5 = graffiti_scribble(img)  # Returns a dictionary with the images mapped to their filename
    # graffiti,ratio = graffiti_writing(img)
    # dmg5[ratio] = graffiti  # Add to the dictionary
    # # Loop through the key,value pairs of dictionary
    # for ratio,dmg in dmg5.items():
    #     dmg_values = calc_quadrant_diff(dmg, img)
    #     values = [float_to_string(round(num, 6)) for num in dmg_values]  # Convert the values into decimal strings
    #     text_file = open(text_base + str(ii) + ".txt", "w")
    #     text_file.write(f"{values[0]} {values[1]} {values[2]} {values[3]}")
    #     # NOTE: graffiti='__' field (may be no longer appropriate)
    #     ii += 1
    #     cv2.imwrite(os.path.join(output_path, class_num + damage_types[5] + "_" + ratio + ".png"), dmg)
    dmgs, attrs = graffiti(img, color=(0,0,0))
    for ii in range(len(dmgs)):
        cv2.imwrite(os.path.join(output_path, class_num + "_" + attrs[ii]["damage"] + "_" + str(attrs[ii]["ratio"]) + ".png"), dmgs[ii])
    
    # BEND
    # dmg6 = bend_vertical(image_path)
    # for degrees,dmg in dmg6.items():
    #     cv2.imwrite(os.path.join(output_path, class_num + damage_types[6] + "_" + degrees + ".png"), dmg)
    # TODO: Write values to file
    dmgs, attrs = bend_vertical(img)
    for ii in range(len(dmgs)):
        cv2.imwrite(os.path.join(output_path, class_num + "_" + attrs[ii]["damage"] + "_" + attrs[ii]["tag"] + ".png"), dmgs[ii])
    
    # GREY
    # dmg7 = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)   # Convert to greyscale
    # # Threshold the image to get a uniform saturation
    # _, dmg7 = cv2.threshold(dmg7, 100, 255, cv2.THRESH_BINARY)
    # cv2.convertScaleAbs(dmg7, dmg7, alpha=1, beta=200)  # No change to contrast, scale brightness
    # dmg7 = cv2.cvtColor(dmg7, cv2.COLOR_GRAY2BGRA)   # Convert back to BGRA to add back the alpha channel
    # dmg7[:,:,3] = alpha_ch
    # cv2.imwrite(os.path.join(output_path, class_num + damage_types[7] + ".png"), dmg7)
    # TODO: Test with exposure_manipulation()
    # TODO: Write values to file
    
    # TODO: CRACKS (thin crack lines across the sign?)
    # TODO: MISSING SECTIONS (missing polygon sections on edges of sign?)


# def float_to_string(num):
#     """Remove trailing zeros and prevent automatic scientific exponent notation."""
#     # https://stackoverflow.com/a/37736333/12350950
#     return ("%.6f" % num).rstrip("0").rstrip(".")

In [ ]:
# Create damaged signs using the images with backgrounds removed
input_path = os.path.join("Traffic_Signs_Templates", "2_Processed_Images")
output_dir = os.path.join("Traffic_Signs_Templates", "3_Damaged_Images")

# Remove any old output and recreate the output directory
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.mkdir(output_dir)

for image_path in load_files(input_path):
    damage_image(image_path, output_dir)

In [ ]:
def add_pole(filename, color):
    """Adds a pole to the imported sign"""
    img = cv.imread(filename, cv.IMREAD_UNCHANGED)
    # Retrieve image dimentions and calculate the new height
    height, width, _ = img.shape  # Discard channel
    new_height = height * 3

    # Create a new blank image with dtype=uint8 (to hold numbers 0-255)
    # Initialise values to 0 so that the extended image is fully transparent
    new_img = np.zeros((new_height, width, 4), dtype=np.uint8)
    # Copy over the original image onto the top portion
    new_img[0:height, :] = img

    # Calculate coordinates
    midpt = width // 2
    offset = width // 40
    x1, x2 = midpt - offset, midpt + offset
    y1, y2 = height, new_height
    # Draw the pole
    cv.rectangle(new_img, (x1, y1), (x2, y2), color, cv.FILLED)

    # Colour any transparent pixels between the sign and the rectangle
    lim = 50  # Max alpha value for the pixel to be considered "transparent"
    margin = int(round( height * 0.9 ))  # Loop over bottom 10% of the sign
    for y in range(margin, height):
        for x in range(x1, x2+1):  # Loop to x2 inclusive
            if new_img[y,x,3] < lim:
                new_img[y,x] = color

    return new_img

In [ ]:
def img_transform(image_path, output_path):
    """Creates and saves different angles of the imported image"""
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    height,width,_ = img.shape

    dst = []
    #0 FORWARD FACING
    dst.append( img )

    #1 EAST FACING
    pts1 = np.float32( [[width/10,height/10], [width/2,height/10], [width/10,height/2]] )
    pts2 = np.float32( [[width/5,height/5], [width/2,height/8], [width/5,height/1.8]] )
    M = cv2.getAffineTransform(pts1,pts2)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #2 NORTH-WEST FACING
    pts3 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts4 = np.float32( [[width*9/10,height/5], [width/2,height/8], [width*9/10,height/1.8]] )
    M = cv2.getAffineTransform(pts3,pts4)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #3 LEFT TILTED FORWARD FACING
    pts5 = np.float32( [[width/10,height/10], [width/2,height/10], [width/10,height/2]] )
    pts6 = np.float32( [[width/12,height/6], [width/2.1,height/8], [width/10,height/1.8]] )
    M = cv2.getAffineTransform(pts5,pts6)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #4 RIGHT TILTED FORWARD FACING
    pts7 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts8 = np.float32( [[width*10/12,height/6], [width/2.2,height/8], [width*8.4/10,height/1.8]] )
    M = cv2.getAffineTransform(pts7,pts8)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #5 WEST FACING
    pts9  = np.float32( [[width/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts10 = np.float32( [[width/9.95,height/10], [width/2.05,height/9.95], [width*9/10,height/2.05]] )
    M = cv2.getAffineTransform(pts9,pts10)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #6 RIGHT TILTED FORWARD FACING
    pts11 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts12 = np.float32( [[width*9/10,height/10], [width/2,height/9], [width*8.95/10,height/2.05]] )
    M = cv2.getAffineTransform(pts11,pts12)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #7 FORWARD FACING W/ DISTORTION
    pts13 = np.float32( [[width/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts14 = np.float32( [[width/9.8,height/9.8], [width/2,height/9.8], [width*8.8/10,height/2.05]] )
    M = cv2.getAffineTransform(pts13,pts14)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #8 FORWARD FACING W/ DISTORTION 2
    pts15 = np.float32( [[width/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts16 = np.float32( [[width/11,height/10], [width/2.1,height/10], [width*8.5/10,height/1.95]] )
    M = cv2.getAffineTransform(pts15,pts16)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #9 FORWARD FACING W/ DISTORTION 3
    pts17 = np.float32( [[width/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts18 = np.float32( [[width/11,height/11], [width/2.1,height/10], [width*10/11,height/1.95]] )
    M = cv2.getAffineTransform(pts17,pts18)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #10 FORWARD FACING W/ DISTORTION 4
    pts19 = np.float32( [[width*9.5/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts20 = np.float32( [[width*9.35/10,height/9.99], [width/2.05,height/9.95], [width*9.05/10,height/2.03]] )
    M = cv2.getAffineTransform(pts19,pts20)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #11 FORWARD FACING W/ DISTORTION 5
    pts21 = np.float32( [[width*9.5/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts22 = np.float32( [[width*9.65/10,height/9.95], [width/1.95,height/9.95], [width*9.1/10,height/2.02]] )
    M = cv2.getAffineTransform(pts21,pts22)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #12 FORWARD FACING W/ DISTORTION 6
    pts23 = np.float32( [[width*9.25/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts24 = np.float32( [[width*9.55/10,height/9.85], [width/1.9,height/10], [width*9.3/10,height/2.04]] )
    M = cv2.getAffineTransform(pts23,pts24)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #13 SHRINK 1
    pts25 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts26 = np.float32( [[width*8/10,height/10], [width*1.34/3,height/10.5], [width*8.24/10,height/2.5]] )
    M = cv2.getAffineTransform(pts25,pts26)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #14 SHRINK 2
    pts27 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts28 = np.float32( [[width*8.5/10,height*3.1/10], [width/2,height*3/10], [width*8.44/10,height*1.55/2.5]] )
    M = cv2.getAffineTransform(pts27,pts28)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #15 FORWARD FACING W/ DISTORTION 7
    pts29 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts30 = np.float32( [[width*8.85/10,height/9.3], [width/1.9,height/10.5], [width*8.8/10,height/2.11]] )
    M = cv2.getAffineTransform(pts29,pts30)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #16 FORWARD FACING W/ DISTORTION 8
    pts31 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts32 = np.float32( [[width*8.75/10,height/9.1], [width/1.95,height/8], [width*8.5/10,height/2.05]] )
    M = cv2.getAffineTransform(pts31,pts32)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #17 FORWARD FACING W/ DISTORTION 9
    pts33 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts34 = np.float32( [[width*8.75/10,height/9.1], [width/1.95,height/9], [width*8.5/10,height/2.2]] )
    M = cv2.getAffineTransform(pts33,pts34)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #18 FORWARD FACING W/ DISTORTION 10
    pts35 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts36 = np.float32( [[width*8.75/10,height/8], [width/1.95,height/8], [width*8.75/10,height/2]] )
    M = cv2.getAffineTransform(pts35,pts36)
    dst.append( cv2.warpAffine(img,M,(width,height)) )
    
    #19 FORWARD FACING W/ DISTORTION 11
    pts37 = np.float32( [[width*9/10,height/10], [width/2,height/10], [width*9/10,height/2]] )
    pts38 = np.float32( [[width*8.8/10,height/7], [width/1.95,height/7], [width*8.8/10,height/2]] )
    M = cv2.getAffineTransform(pts37,pts38)
    dst.append( cv2.warpAffine(img,M,(width,height)) )

    # Retrieve the filename to save as
    _,tail = ntpath.split(image_path)  # Filename of the image, parent directories removed
    title,_ = tail.rsplit('.', 1)      # Discard extension
    
    # Save the transformed images
    for ii in range(1, len(dst)):
        save_path = os.path.join(output_path, title)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        cv2.imwrite(os.path.join(save_path, str(ii) + ".png"), dst[ii])
#    plt.show()

In [ ]:
input_path = os.path.join("Traffic_Signs_Templates", "3_Damaged_Images")
output_dir = os.path.join("Traffic_Signs_Templates", "4_Transformed_Images")

# Loop through each folder in the input directory
for folder_path in load_paths(input_path):
    # Retrieve the sign number to create a directory
    _,number = os.path.split(folder_path)
    save_dir = os.path.join(output_dir, number)
    
    for image_path in load_files(folder_path):
        img_transform(image_path, save_dir)

In [ ]:
def to_png(directory):
    """Convert all files in 'directory' to PNG images"""
    for files in load_paths(directory):
        title, extension = files.split('.')
        img = Image.open(files).convert('RGBA')
        if (not extension == "png"):
            os.remove(files)
        img.save(title + ".png")

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

for bg_folders in load_paths("Backgrounds"):
    to_png(bg_folders)

In [ ]:
def find_image_exposure(paths, channels):
    exposures = []
    for image_path in paths:
        img_grey = Image.open(image_path).convert('LA')  # Greyscale with alpha
        img_rgba = Image.open(image_path)
        
        stat1 = ImageStat.Stat(img_grey)
        # Average pixel brighness
        avg = stat1.mean[0]
        # RMS pixel brighness
        rms = stat1.rms[0]
        
        stat2 = ImageStat.Stat(img_rgba)
        # Consider the number of channels
        # Background may have RGB while traffic sign has RGBA
        if (channels == 3):
            # Average pixels preceived brightness
            r,g,b = stat2.mean
            avg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))
            # RMS pixels perceived brightness
            r,g,b = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))
        else:
            # Average pixels preceived brightness
            r,g,b,a = stat2.mean
            avg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))
            # RMS pixels perceived brightness
            r,g,b,a = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2)) 

        exposures.append( [image_path,avg,rms,avg_perceived,rms_perceived] )

    return exposures     

In [ ]:
def exposure_manipulation(signs_paths, backgrounds_paths):
    background_exposures = find_image_exposure(background_paths, 4)
    signs_exposures = find_image_exposure(signs_paths, 4)
    
    for ii in range(0,len(background_paths)):
        print("Processed: " + str(float(ii) / float(len(background_paths)) * 100) + " %")
        
        img = Image.open(background_exposures[ii][0])

        for sign_path in signs_paths:        
            dirc, sub, el = dir_split(background_exposures[ii][0])
            title, extension = el.rsplit('.', 1)

            parent_dir, sub_dir, folder, folder2, element = dir_split(sign_path)
            head, tail = element.rsplit('.', 1)

            
            ###   ORIGINAL EXPOSURE IMPLEMENTATION   ###
            brightness_avrg = 1.0
            brightness_rms = 1.0
            brightness_avrg_perceived = 1.0
            brightness_rms_perceived = 1.0
            brightness_avrg2 = 1.0
            brightness_rms2 = 1.0

            # abs(desired_brightness - actual_brightness) / abs(brightness_float_value) = ratio
            avrg_ratio = 11.0159464507

            rms_ratio = 8.30320014372

            percieved_avrg_ratio = 3.85546373056

            percieved_rms_ratio = 35.6344530649

            avrg2_ratio = 1.20354549572

            rms2_ratio = 40.1209106864

            peak1 = Image.open(sign_path).convert('LA')
            peak2 = Image.open(sign_path).convert('RGBA')

            stat = ImageStat.Stat(peak1)
            avrg = stat.mean[0]
            rms = stat.rms[0]

            
            ### IMAGE MANIPULATION MAIN CODE STARTS ###

            # MINIMISE MARGIN BASED ON AVERAGE FOR TWO CHANNEL BRIGNESS VARIATION
            margin = abs(avrg - float(background_exposures[ii][1]))
            
            brightness_avrg = margin / avrg_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright = enhancer.enhance(brightness_avrg)
            stat = ImageStat.Stat(avrg_bright)
            avrg = stat.mean[0]

            
            # MINIMISE MARGIN BASED ON ROOT MEAN SQUARE FOR TWO CHANNEL BRIGNESS VARIATION
            margin = abs(rms - float(background_exposures[ii][2]))

            brightness_rms = margin / rms_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright = enhancer.enhance(brightness_rms)
            stat = ImageStat.Stat(rms_bright)
            rms = stat.rms[0]

            
            # MINIMISE MARGIN BASED ON AVERAGE FOR RGBA ("PERCEIVED BRIGHNESS")
            # REFERENCE FOR ALGORITHM USED: http://alienryderflex.com/hsp.html
            stat2 = ImageStat.Stat(peak2)
            r, g, b, a = stat2.mean
            avrg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))
            margin = abs(avrg_perceived - float(background_exposures[ii][3]))
            
            brightness_avrg_perceived = margin / percieved_avrg_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright_perceived = enhancer.enhance(brightness_avrg_perceived)
            stat2 = ImageStat.Stat(avrg_bright_perceived)
            r, g ,b, a = stat2.mean
            avrg_perceived = math.sqrt(0.241 * (r**2) + 0.691 * (g**2) + 0.068 * (b**2))        


            # MINIMISE MARGIN BASED ON RMS FOR RGBA ("PERCEIVED BRIGHNESS")
            # REFERENCE FOR ALGORITHM USED: http://alienryderflex.com/hsp.html
            r, g, b, a = stat2.rms
            rms_perceived = math.sqrt(0.241 * (r**2) + 0.691 * (g**2) + 0.068 * (b**2))

            margin = abs(rms_perceived - float(background_exposures[ii][4]))

            brightness_rms_perceived = margin / percieved_rms_ratio 

            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright_perceived = enhancer.enhance(brightness_rms_perceived)
            stat2 = ImageStat.Stat(rms_bright_perceived)
            r, g, b, a = stat2.rms
            rms_perceived = math.sqrt(0.241 * (r**2) + 0.691 * (g**2) + 0.068 * (b**2))        

            
            stat3 = ImageStat.Stat(peak2)
            avrg2 = stat3.mean[0]
            rms2 = stat3.rms[0]

            """
            #FUSION OF THE TWO AVERAGING METHODS
            margin = abs(avrg2-float(background_exposures[ii][1]))
            brightness_avrg2 = margin/avrg2_ratio 
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright2 = enhancer.enhance(brightness_avrg2)
            stat3 = ImageStat.Stat(avrg_bright2)
            avrg2 = stat3.mean[0]       
            """
            
            
            """
            #FUSION OF THE TWO RMS METHODS
            margin = abs(rms2-float(background_exposures[ii][2]))
            brightness_rms2 = margin/rms2_ratio 
            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright2 = enhancer.enhance(brightness_rms2)
            stat3 = ImageStat.Stat(rms_bright2)
            rms2 = stat3.rms[0]
            """
            
            avrg_bright = avrg_bright.resize((150,150), Image.ANTIALIAS)
            rms_bright = rms_bright.resize((150,150), Image.ANTIALIAS)
            avrg_bright_perceived = avrg_bright_perceived.resize((150,150), Image.ANTIALIAS)
            rms_bright_perceived = rms_bright_perceived.resize((150,150), Image.ANTIALIAS)
            # avrg_bright2 = avrg_bright2.resize((150,150), Image.ANTIALIAS)
            # rms_bright2 = rms_bright2.resize((150,150), Image.ANTIALIAS)

            
            exp_dir = "Traffic_Signs_Exposure_Manipulation"
            avrg_bright.save(os.path.join(exp_dir,sub,title,"SIGN_"+folder,folder2,head+"_AVERAGE."+tail))
            rms_bright.save(os.path.join(exp_dir,sub,title,"SIGN_"+folder,folder2,head+"_RMS."+tail))
            avrg_bright_perceived.save(os.path.join(exp_dir,sub,title,"SIGN_"+folder,folder2,head+"_AVERAGE_PERCEIVED."+tail))
            rms_bright_perceived.save(os.path.join(exp_dir,sub,title,"SIGN_"+folder,folder2,head+"_RMS_PERCEIVED."+tail))
            # avrg_bright2.save(os.path.join(exp_dir,sub,title,"SIGN_"+folder,folder2,head+"_AVERAGE2."+tail))
            # rms_bright2.save(os.path.join(exp_dir,sub,title,"SIGN_"+folder,folder2,head+"_RMS2."+tail))
    
    print("Processed: " + str(100) + " %")
    print("Process was successful")

In [ ]:
def fade_manipulation(signs_paths, backgrounds_paths):
    background_exposures = find_image_exposure(background_paths, 4)
    signs_exposures = find_image_exposure(signs_paths, 4)
    
    print("Processed: 0.0 %")
    ii = 0
    prev = 0
    for sign_path in signs_paths:
        progress = float(ii) / float(len(signs_paths)) * 100
        if progress >= prev + 5: #Prevent spamming of progress prints
            prev = prev + 5
            print("Processed: " + str(progress) + " %")

        dirc, sub, el = dir_split(background_exposures[0][0])
        title, extension = el.split('.')

        parent_dir, sub_dir, folder, folder2, element = dir_split(sign_path)
        head, tail = element.split('.')

        img = cv2.imread(sign_path, cv2.IMREAD_UNCHANGED)


        ###   GRADUAL FADE IMPLEMENTATION   ###
        # Retrieve alpha data from original image
        splitImg = cv2.split(img)
        if len(splitImg) == 4:
            alphaData = splitImg[3]

        for jj in range(0,5):
            dmg6 = img.copy()
            alpha = 1 - (jj * 0.19)
            beta = (jj + 1) * 40
            if jj > 0:
                cv2.convertScaleAbs(img, dmg6, alpha, beta) # Scale the contrast and brightness
                dmg6[:, :, 3] = alphaData

            dmg6 = cv2.resize(dmg6, (150,150))
            fad_dir = "Traffic_Signs_Fade_Manipulation"
            cv2.imwrite(os.path.join(fad_dir,"SIGN_"+folder,folder2,head+"_FADE-"+str(jj)+"."+tail), dmg6)
        ii = ii + 1
    
    
    print("Processed: " + str(100) + " %")
    print("Process was successful")

In [ ]:
bg_dir = "Backgrounds"
transform_dir = os.path.join("Traffic_Signs_Templates", "4_Transformed_Images")

for dirs in load_paths(bg_dir):
    sep = os.sep  # Directory separator: '/' or '\'

    if original is True:
        for background in load_paths(dirs):
            iniitial, subd, element = dir_split(background)
            title, extension = element.split('.')

            for signp in load_paths(transform_dir):
                for sign in load_paths(signp):
                    d,s,f,e = dir_split(sign)  # Eg. s = 4_Transformed_Images, f = 9, e = 9_BOTTOM_HOLE

                    exp_dir = "Traffic_Signs_Exposure_Manipulation" + sep
                    if (not os.path.exists(exp_dir + subd + sep + title + sep + "SIGN_" + f + sep + e)):
                        os.makedirs(exp_dir + subd + sep + title + sep + "SIGN_" + f + sep + e)
    else:
        for signp in load_paths(transform_dir):
            for sign in load_paths(signp):
                d,s,f,e = dir_split(sign)

                fade_dir = "Traffic_Signs_Fade_Manipulation" + sep
                if (not os.path.exists(fade_dir + "SIGN_" + f + sep + e)):
                    os.makedirs(fade_dir + "SIGN_" + f + sep + e)

signs_paths = []
for p in load_paths(transform_dir):
    for d in load_paths(p):
        signs_paths += load_paths(d)

background_paths = []  # Load the paths of the background images into a single list
for subfolder in load_paths(bg_dir):
    background_paths += load_paths(subfolder)

if original is True:
    exposure_manipulation(signs_paths, background_paths)
else:
    fade_manipulation(signs_paths, background_paths)

In [ ]:
def avrg_pixel_rgb(image, chanels):
    stat = ImageStat.Stat(image)
    if (chanels == 4):
        r,g,b,a = stat.rms
    else:
        r,g,b = stat.rms
    
    return [r,g,b]

In [ ]:
def find_bw_images(directory):
    images = []
    for sign in load_paths(directory):
        for damage in load_paths(sign):
            img = Image.open(damage).convert('RGBA')
            rgb = avrg_pixel_rgb(img, 4)
            rg = abs(rgb[0] - rgb[1])
            rb = abs(rgb[0] - rgb[2])
            gb = abs(rgb[1] - rgb[2])
            
            temp = dir_split(damage)
            list = temp[-1].split('.')
            if len(list) == 2:
                head = list[0]
            else:
                head = list[0] + '.' + list[1]
                    
            if (rg <= 1 and rb <= 1 and gb <= 1):
                images.append(head)
    return images

In [ ]:
def find_useful_signs(directory):
    """Removes bad signs, such as those which are all white or all black"""
    bw_images = find_bw_images(os.path.join("Traffic_Signs_Templates", "3_Damaged_Images"))
    for background_dir in load_paths(directory):
        for background in load_paths(background_dir):
            for signs in load_paths(background):
                for dmgs in load_paths(signs):
                    temp = []
                    for imgs in load_paths(dmgs):
                        temp.append(imgs)
                    exposures = find_image_exposure(temp,4)
                    
                    ii = 0
                    for images in load_paths(dmgs):
                        #Find brightness
                        img = Image.open(images).convert('RGBA')

                        rgb = avrg_pixel_rgb(img,4)
                        rg = abs(rgb[0] - rgb[1])
                        rb = abs(rgb[0] - rgb[2])
                        gb = abs(rgb[1] - rgb[2])

                        is_bw = False

                        for s in bw_images:
                            if s in exposures[ii][0]:
                                is_bw = True

                        if (rg <= 16 and rb <= 16 and gb <= 16):
                            if (not is_bw):
                                os.remove(images)
                            #Threshold values for black and white images
                            elif (rgb[0] < 70 and rgb[1] < 70 and rgb[2] < 70):
                                os.remove(images)
                            elif (rgb[0] > 155 and rgb[1] > 155 and rgb[2] > 155):
                                os.remove(images)

                        elif (not is_bw):
                            #Delete light blue images
                            if(rgb[2] > rgb[0] and rgb[2] >= rgb[1]):
                                if (gb <= 10):
                                    os.remove(images)
                    ii += 1

In [ ]:
if original is True:
    directory = "Traffic_Signs_Exposure_Manipulation"
    find_useful_signs(directory)

In [ ]:
def insert_poisson_noise (image):
    vals = len(np.unique(image))
    vals = 2.05 ** np.ceil(np.log2(vals))
    noisy = np.random.poisson(image * vals) / float(vals)
    return noisy

In [ ]:
def insert_Gaussian_noise (image):
    row,col,ch= image.shape
    mean = 0
    var = 0.5
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy

In [ ]:
def insert_speckle_noise (image):
    row,col,ch = image.shape
    gauss = np.random.randn(row,col,ch)
    gauss = gauss.reshape(row,col,ch)        
    noisy = image + image * gauss
    return noisy

In [ ]:
def random_noise_method (image):
    """
    i = random.randint(1, 3)
    if (i == 1):
        return insert_poisson_noise(image)
    elif (i==2):
        return insert_Gaussian_noise(image)
    else:
        return insert_speckle_noise(image)
    """
    image.setflags(write=1)
    # Add noise in every pixel with random probability 0.4
    for im in image:
        px = 0
        for pixel in im:
            apply_noise = random.randint(0,100)
            if apply_noise > 40:
                # RGB values
                r = pixel[0]
                g = pixel[1]
                b = pixel[2]
                a = pixel[3]
                # Find current relative lumination for brighness
                # Based on: https://en.wikipedia.org/wiki/Relative_luminance
                relative_lumination = 0.2126*r + 0.7152*g + 0.0722*b
                # Find differences between RGB values     
                rg = False
                r_to_g = float(r) / float(g)
                if (r_to_g >= 1):
                    rg = True
                
                rb = False
                r_to_b = float(r) / float(b)
                if (r_to_b >= 1):
                    rb = True
                
                gb = False
                g_to_b = float(g) / float(b)
                if (g_to_b >= 1):
                    gb = True
                
                equal = False
                if (r == g == b):
                    equal = True

                # In order to determine the margin in which the new brighness
                # should be within, the upper and lower limits need to be found
                # The Relative luminance in colorimetric spaces has normalised
                # values between 0 and 255
                upper_limit = 255
                lower_limit = 0
                if (relative_lumination + 40 < 255):
                    upper_limit = relative_lumination + 40
                if (relative_lumination - 40 > 0):
                    lower_limit = relative_lumination - 40

                # Compute new brightness value
                new_lumination = random.randint(int(lower_limit), int(upper_limit))

                # Find the three possible solutions that satisfy
                # -> The new lumination chosen based on the Relative luminance equation
                # -> The precentages computed between every rgb value

                solutions = []

                for r in range(1,255):
                    for g in range(1,255):
                        for b in range(1,255):
                            rg = False
                            r_to_g = float(r) / float(g)
                            if (r_to_g >= 1):
                                rg = True
                            
                            rb = False
                            r_to_b = float(r) / float(b)
                            if (r_to_b >= 1):
                                rb = True
                            
                            gb = False
                            g_to_b = float(g) / float(b)
                            if (g_to_b >= 1):
                                gb = True
                            
                            e = False
                            if(r == g == b):
                                e = True
                            
                            if (0.2126*r + 0.7152*g + 0.0722*b == 100) and rg==rg and rb==rb and gb==GB and e==equal:
                                solutions.append([r,g,b])

                # Find the solution that precentage wise is closer to the original
                # difference between the values
                percentages = []

                for solution in solutions:
                    r = solution[0]
                    g = solution[1]
                    b = solution[2]
                    percentages.append((float(r) / float(g)) + (float(r) / float(b)) + (float(g) / float(b)))

                ii = 0
                pos = 0
                best = percentages[0]
                for p in percentages[1:]:
                    if p < best:
                        pos = ii
                    ii += 1

                # Assign new pixel values
                im[px] = [solutions[pos][0], solutions[pos][1], solutions[pos][2], A]
            px += 1
            
    return image

In [ ]:
def has_opaque_pixel(line):
    """Checks if a line of pixels contains a pixel above a transparency threshold"""
    opaque = False
    for pixel in line:
        # Check if pixel is opaque
        if pixel[3] > 200:
            opaque = True
            break  # Stop searching if one is found
    return opaque

def bounding_axes(img):
    """Returns the bounding axes of an image with a transparent background"""
    # Top axis
    y_top = 0
    for row in img:  # Iterate through each row of pixels, starting at top-left
        if has_opaque_pixel(row) is False:  # Check if the row has an opaque pixel
            y_top += 1  # If not, move to the next row
        else:
            break  # If so, break, leaving y_top as the bounding axis

    # Bottom axis
    height = img.shape[0]
    y_bottom = height - 1
    for row in reversed(img):  # Iterate from the bottom row up
        if has_opaque_pixel(row) is False:
            y_bottom -= 1
        else:
            break

    # Left axis
    # Rotate 90 degrees to iterate through what were originally columns
    r_img = imutils.rotate_bound(img, 90)
    x_left = 0
    for column in r_img:
        if has_opaque_pixel(column) is False:
            x_left += 1
        else:
            break

    # Right axis
    r_height = r_img.shape[0]
    x_right = r_height - 1
    for column in reversed(r_img):
        if has_opaque_pixel(column) is False:
            x_right -= 1
        else:
            break

    # FOR TESTING
    # img[y_top, :] = (255, 0, 0, 255)
    # img[y_bottom, :] = (255, 0, 0, 255)
    # img[:, x_left] = (255, 0, 0, 255)
    # img[:, x_right] = (255, 0, 0, 255)
    # cv2.imwrite(image_dir, img)

    return [x_left, x_right, y_top, y_bottom]

# FOR TESTING
# for img in load_paths("Traffic_Signs_Templates/4_Transformed_Images/0/0_ORIGINAL"):
#     bounding_axes(img)

In [ ]:
### FULL BACKGROUND ###
def new_data(image_dir, bg_dir, label_file, filename, values):
    """Blends synthetic signs with backgrounds"""
    bg = cv2.imread(bg_dir, cv2.IMREAD_UNCHANGED)
    fg = cv2.imread(image_dir, cv2.IMREAD_UNCHANGED)
    bg_height, bg_width, _ = bg.shape
    fg_height, fg_width, _ = fg.shape

    # Rescaling the sign to correct its size relative to the background
    # NOTE: Assumes background aspect ratio somewhat close to 16:9 (1.778)
    current_ratio = fg_width / bg_width # Ratio of sign width to the background width
    target_ratio = random.uniform(0.033, 0.066) # Aiming for between 3.3% and 6.6% of bg width
    scale_factor = target_ratio / current_ratio
    new_size = int(fg_width * scale_factor)
    fg = cv2.resize(fg, (new_size, new_size))

    # Randomise sign placement within middle third of background
    x = random.randint(0, bg_width - fg_width)
    third = bg_height // 3
    y = random.randint(third, bg_height - third)

    # Build label
    axes = bounding_axes(fg)  # Retrieve bounding axes of the sign image
    axes[0] += x  # Adjusting bounding axis to make it relative to the whole bg image
    axes[1] += x
    axes[2] += y
    axes[3] += y
    bounds = str(axes[0]) + " " + str(axes[1]) + " " + str(axes[2]) + " " + str(axes[3])

    # It is assumed that the final .jpg -> .png conversion step is executed
    label = filename + ".jpg " + bounds + " " + values + "\n"
    label_file.write(label)
    image = overlay(fg, bg, x, y)

    return image


### ORIGINAL ### (Outdated - used for creating classification data)
# def new_data(image_dir,bg_dir): #Blends synthetic signs with backgrounds
#     # Import background image
#     background_img_raw = Image.open(bg_dir).convert('RGBA')  
#     background_img_raw = background_img_raw.resize((150,150), Image.ANTIALIAS)
#     background_img = np.array(background_img_raw)  
#     background_img_float = background_img.astype(float)  

#     # Import foreground image
#     foreground_img_raw = Image.open(image_dir)  
#     foreground_img = np.array(foreground_img_raw)  
#     foreground_img_float = foreground_img.astype(float)  

#     # Blend images
#     opacity = 1  
#     blended_img_float = blend_modes.grain_merge(background_img_float, foreground_img_float, opacity)

#     # Convert blended image back into PIL image
#     blended_img = np.uint8(blended_img_float)
#     blended_img_raw = Image.fromarray(blended_img)  
    
#     foreground_img_raw = foreground_img_raw.resize((149,149), Image.ANTIALIAS)
#     blended_img_raw.paste(foreground_img_raw, (0, 0), foreground_img_raw)
#     blended_img_raw = blended_img_raw.resize((48,48), Image.ANTIALIAS)
    
#     # temp = np.uint8(blended_img_raw)
#     # temp = random_noise_method(temp)
    
#     # blended_img_raw = Image.fromarray(np.uint8(temp)) 
    
#     return blended_img_raw

In [ ]:
# Create the required directories in SGTSD
if (not os.path.exists("SGTSD/Images")):
    # Numbered Version
    for sign in load_paths(os.path.join("Traffic_Signs_Templates", "1_Input_Images")):  # Dir. for each sign type
        head, tail = sign.split('.')
        name = dir_split(head)
        os.makedirs(os.path.join("SGTSD", "Images", name[-1]))
        j = 0
        for dmg in range(len(damage_types)):  # Dir. for each damage type
            os.makedirs(os.path.join("SGTSD", "Images", name[-1], name[-1] + "_" + str(j)))
            j = j + 1
    
    # Named Version (Outdated?)
#     for sign in load_paths(os.path.join("Traffic_Signs_Templates", "1_Input_Images")):  # Dir. for each sign type
#         head, tail = sign.split('.')
#         name = dir_split(head)
#         os.makedirs(os.path.join("SGTSD", "Images", name[-1]))
#         for dmg in load_paths(os.path.join("Traffic_Signs_Templates", "3_Damaged_Images")): #Dir. for each damage type
#             headD,tailD = dmg.split('.')
#             nameD = dir_split(headD)
#             os.makedirs(os.path.join("SGTSD", "Images", name[-1], nameD[-1]))

In [ ]:
# Create a README file
content = '''
-----------------------------------------------
|                     -*-                     |
|Synthetically Generated Traffic Sign Dataset |
|                     -*-                     |
-----------------------------------------------

This directory contains the generated training
set to be used for training a Convolutional
Neural Network (CNN).

It may be used for any detector desired and it
is not limited to a specific set of traffic
sign templates.
 

----------------------------------------------
Content
----------------------------------------------

The number of examples is based on the number:
->of traffic signs that were used as templates
->of damage types applied to images
->of transformations in image manipulations
->of the brightness variation values used
->of blending procedures


----------------------------------------------
Image format and naming
----------------------------------------------

The images created are of "jpg" format
with RGBA channels.

   XXX/XXX_YYY/XXX_YYY_ZZZ.jpg

(X) is used to distinguish the sign class, (Y)
is used to distinguish the damage type and (Z)
is used to indicate the example number.


----------------------------------------------
Additional information
----------------------------------------------

Contact Email:
   Original Code:
	   asterga@essex.ac.uk

   Adapted Damage Code:
      kristian.rados@student.curtin.edu.au
      seana.dale@student.curtin.edu.au


----------------------------------------------
'''

text_file = open("SGTSD/Readme_Images.txt", "w")
text_file.write(content)
text_file.close()

In [ ]:
# List of paths for all SGTSD relevant files using exposure_manipulation
def create_paths_list(imgs_directory, bg_directory):
    directories = []
    for places in load_paths(imgs_directory):  # List of places: originally either UK_rural or UK_urban
        for imgs in load_paths(places):  # Folder for each bg image: eg. IMG_0
            dr = dir_split(imgs)
            bg = os.path.join(bg_directory, dr[-2], dr[-1] + ".png")  # Retrieving relevant bg image
            for signs in load_paths(imgs):  # Folder for each sign type: eg. SIGN_9
                for dmgs in load_paths(signs):  # Folder for each damage type: eg. 0_HOLES
                    for png in load_paths(dmgs):
                        directories.append([png, bg])
    return directories  # Directory for every single FILE and it's relevant bg FILE

In [ ]:
# List of paths for all SGTSD relevant files using fade_manipulation; backgrounds are assigned to 
def create_assigned_paths_list(imgs_directory, bg_directory): #TODO: is this the same as above?
    directories = []
    for places in load_paths(bg_directory):  # Folder for each place: eg. GTSDB
        for imgs in load_paths(places):  # Iterate through each b.g. image: eg. IMG_0
            for signs in load_paths(imgs_directory):  # Folder for each sign type: eg. SIGN_9
                for dmgs in load_paths(signs):  # Folder for each damage type: eg. 9_HOLES
                    for png in load_paths(dmgs):
                        directories.append([png, imgs])
    return directories  # Directory for every single FILE and its relevant bg FILE

In [ ]:
if original is True:
    directories = create_paths_list("Traffic_Signs_Exposure_Manipulation", "Backgrounds")
else:
    directories = create_assigned_paths_list("Traffic_Signs_Fade_Manipulation", "Backgrounds")
print("Files to be generated: " + str(len(directories)))

In [ ]:
# Paths of images needed to generate examples for 'sign' with damage 'dmg'
def list_for_sign_x(sign, dmg, directories):
    l = []
    for elements in directories:
        foreground = dir_split(elements[0])
        if (foreground[-2] == sign + dmg):  # Eg. if (9_YELLOW == 4_ORIGINAL)
            l.append(elements)
    return l  # Directory for every single sign and its relevant background image

In [ ]:
#TODO: Something has gone wrong here onwards... final_directories is filled with empty elements

In [ ]:
final_directories = []  # Reformat list to have each sign and damage as their own dimensions
signs = load_paths(os.path.join("Traffic_Signs_Templates", "1_Input_Images"))
for i in signs:
    head, tail = ntpath.split(i)
    sign, extension = tail.split('.')  # Eg. sign == "9"

    sign_list = []  # List of damages, which are each list of signs
    for dmg in damage_types:  # Damage_types is from 'def damage_images:' cell
        sign_list.append(list_for_sign_x(sign, dmg, directories))
    final_directories.append(sign_list)  # List of types -> lists of damages -> lists of signs

In [ ]:
# Generate and write the new data to its file
direct = os.path.join("SGTSD", "Images")
direct1 = os.path.join("SGTSD", "Labels")
folders = load_paths(direct)
n = []  # Numbers from the folder names for the signs
for folder in folders:
    head, tail = ntpath.split(folder)
    n.append(tail)

# Count how many signs there are; needed for the progress bar
total = 0
for signs in final_directories:
    for damages in signs:
        for dirs in damages:
            total += 1

count = 0
ii = 0
for signs in final_directories:  # Iterate through sign types
    jj = 0
    for damages in signs:  # Iterate through damage types
        # FIXME: For some reason secondary progress counter has broken from initial implementation
        print("Processed: " + str(float(count) / float(total) * 100) + " %")
        kk = 0

        label_filename = os.path.join(direct1, n[ii], n[ii] + "_" + str(ij) + ".txt")
        text_file = open(label_filename, "r")
        values = text_file.read()  # Retrieve the damage values created earlier in the damage_images function
        text_file = open(label_filename, "w")

        for dirs in damages:  # dirs == the foreground and background images for one generated sign
            filename = os.path.join(direct, n[ii], n[ii] + "_" + str(ij), n[ii] + "_" + str(ij) + "_" + str(kk))
            image = new_data(dirs[0], dirs[1], text_file, filename, values) # Combining b.g. with sign f.g.
            cv2.imwrite(filename + ".png", image)

            count += 1
            kk += 1
        jj += 1
        text_file.close()
    ii += 1
print("Processed: " + str(100) + " %")  # FIXME: changing damage names broke something

In [ ]:
string = '''
-------------------------------------
BREAKDOWN OF FILES GENERATED BY CLASS
-------------------------------------
'''
total = 0
for i in range(len(final_directories)):
    current = 0
    for j in range(len(final_directories[i])):
        current = current + len(final_directories[i][j])
    s = "Generated " + str(current) + " examples for sign class " + str(i + 1)
    string = string + '\n' + s + '\n'
    total = total + current
string = string + '\n' + "TOTAL: " + str(total) + '\n' + "Generated on: " + datetime.now().strftime("%Y-%m-%d %H:%M") + '\n'
string = string + "-------------------------------------"
text_file = open(os.path.join("SGTSD", "generated_images_about.txt"), "w")
text_file.write(string)
text_file.close()

In [ ]:
def png_to_jpeg(filepath):
    sep = os.sep

    dirs = dir_split(filepath)
    title,extension = dirs[-1].split('.')
    del dirs[-1]
    string = sep.join(dirs)
    string = os.path.join(string, title + ".jpg")
    png = Image.open(filepath)
    png.load()  # Required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3])  # 3 is the alpha channel
    background.save(string, 'JPEG', quality=100)
    os.remove(filepath)

In [ ]:
dirs = direct = os.path.join("SGTSD", "Images")
i = 1
for path in load_paths(dirs):
    print("Processed: " + str(float(i - 1) / float(len(final_directories)) * 100) + " %")
    for damage in load_paths(path):
        for image in load_paths(damage):
            if (image.endswith("png")):
                png_to_jpeg(image)
    i += 1
print("Processed: " + str(100) + " %")

In [ ]:
# shutil.rmtree("Traffic_Signs_Exposure_Manipulation")

In [ ]:
# shutil.rmtree("Traffic_Signs_Fade_Manipulation")

In [ ]:
# shutil.rmtree("Traffic_Signs_Templates/4_Transformed_Images")
# shutil.rmtree("Traffic_Signs_Templates/3_Damaged_Images")
# shutil.rmtree("Traffic_Signs_Templates/2_Processed_Images")

In [ ]:
# shutil.rmtree("SGTSD")  # Be careful with this one